# Colin test case

In [ ]:
from pathlib import Path
import os
import sys
import pandas
import numpy
from trustutils import run

run.introduction("François PECQUERY", "30/04/2024")
run.TRUST_parameters()

## Simulations setup
All parameters to be filled by the user are provided in the following cell.
The user can choose the turbulence model: k-omega or k-omega with the Sato model (1981) or k-omega with the Antoine du Cluzeau model.

In [ ]:
# Can be changed
force_computation = True
number_of_partitions = 6
save_figures = True
turbulence_choice = ["komega-sato"]
#turbulence_choice = ["komega", "komega-sato", "komega-hzdr"]
subset = [1,4,9,15] #list(range(0, 17)) 

dparam = {}
dparam["tmax"] = 0.1 #10
dparam["nb_pas_dt_max"] = 1000000000
dparam["seuil_statio"] = 1e-4
dparam["solver"] = "ice"
dparam["facsec"] = 1
dparam["max_facsec"] = 1
refinement_levels = [1, 2] # two mesh levels

# Shouldn't be changed
dataroot = "jdd"
rho_l = 998.30
rho_g = 1.2
grav = -9.81

## Prepare the simulations directory

In [ ]:
if force_computation or not os.path.exists('build'):
    run.reset()
    run.initBuildDirectory()
build = run.BUILD_DIRECTORY

## Retrieve experimental data and correct them.

In [ ]:
######################
# Read experimental data
trial_matrix = pandas.read_csv("data_exp/Colin2012.csv")

# number of tests
n_exp = len(trial_matrix["SetNumber"])

# Liquid volume fraction
trial_matrix["alpha_l0"] = trial_matrix['JF']/(trial_matrix['JF']+trial_matrix['JG'])

# Gas volume fraction / void fraction
trial_matrix["alpha_g0"] = trial_matrix['JG']/(trial_matrix['JF']+trial_matrix['JG'])

# Liquid velocity
trial_matrix["u_0"] = trial_matrix['JF']/trial_matrix["alpha_l0"]

# Boundary condition for turbulent kinetic energy
trial_matrix["CL_k"] = 0.01*trial_matrix["u_0"]**2

# Boundary condition for omega
trial_matrix["CL_om"] = trial_matrix["u_0"]/trial_matrix["D_h"]

# gravity
for ii, setn in enumerate(trial_matrix["SetNumber"]):
    expNumber = setn
    if (expNumber[:2] == "U_") :
        trial_matrix["gravity"] = -9.81
    if (expNumber[:2] == "D_") :
        trial_matrix["gravity"] = 9.81

# Fluid couple
trial_matrix["fluid"] = "water-air"

# Number of meshes
trial_matrix["mesh"] = "Colin"

# Height of the test set
trial_matrix["height"] = 3.08 #77*0.04

Available keys in trial_matrix

In [ ]:
trial_matrix.keys()

## Calculate the bublle diameters

In [ ]:
# Function to create a predefined field of bubble diameters

# degree of interpolation
nfitd = 4

def str_r_loc_d_loc(rl, dl):
    r_l_l = []
    d_l_l = []
    for ii, val in enumerate(rl):
        if (numpy.isnan(dl[ii]) == False):
            r_l_l += [val]
            d_l_l += [dl[ii]]
            
    tab_rmin_max = [0, 0]
    tab_dmin_max = [0, 0]
            
    tab_polyfitd = numpy.polyfit(r_l_l, d_l_l, nfitd)
    tab_rmin_max[0] = (r_l_l[0] + r_l_l[1] )/2.
    tab_rmin_max[1] = (r_l_l[-1] + r_l_l[-2])/2.
    tab_dmin_max[0] = (d_l_l[0] + d_l_l[1] )/2.
    tab_dmin_max[1] = (d_l_l[-1] + d_l_l[-2])/2.

    str_diam = "0."

    # First we take care of what happens above and under the highest values
    str_diam += f"+((X*X+Y*Y)<({tab_rmin_max[0]}*{tab_rmin_max[0]}))"
    str_diam += f"*{tab_dmin_max[0]}"
    str_diam += f"+((X*X+Y*Y)]({tab_rmin_max[1]}*{tab_rmin_max[1]}))"
    str_diam += f"*{tab_dmin_max[1]}"

    # Then we take care of the middle
    str_loc = "0."
    for ii in range(nfitd + 1):
        str_loc += f"+({tab_polyfitd[ii]})"
        for jj in range(nfitd - ii):
            str_loc+= "*sqrt(x*x+y*y)"
            
    str_diam += f"+((X*X+Y*Y)]({tab_rmin_max[0]}*{tab_rmin_max[0]}))" # put to zero outside the right interval
    str_diam += f"*((X*X+Y*Y)<({tab_rmin_max[1]}*{tab_rmin_max[1]}))" 
    str_diam += f"*({str_loc})"          
    str_diam = f"1.e-3*({str_diam})" # mm => m
    return str_diam

tab_str_diam = ["1.e-3"]*len(trial_matrix["SetNumber"]) # Default single phase
for index, row in trial_matrix.iterrows():
    expNumber = row["SetNumber"]
    path = f"data_exp/{expNumber}_G.txt"
    print(f" index is {index} and expe number is {expNumber}")

    if os.path.exists(path):
        tab_G = pandas.read_csv(path,sep = '\t',skiprows = [0,1,2,4])
        r_loc = numpy.array(tab_G["r+"])*row['D_h']/2.
        if ("d32e" in tab_G) :
            d_loc = numpy.array(tab_G["d32e"])# These are in mm
        elif ("d" in tab_G) :
            d_loc = numpy.array(tab_G["d"]) # These are in mm
        else :
            print('PROBLEM !')
        tab_str_diam[index] = str_r_loc_d_loc(r_loc, d_loc)
        
trial_matrix["Dbubble"] = tab_str_diam

Creation of Substitution Dictionaries

In [ ]:
# Modèles
def dico_model():
    dico = {}
    dico["interface"] = "interface_eau_air interface_sigma_constant { tension_superficielle 0.0728 }"
    dico["carrying_phase"] = "liquide_eau Fluide_Incompressible { mu champ_uniforme 1 1.002e-3 rho champ_uniforme 1 998.30 lambda Champ_Uniforme 1 0.604 Cp Champ_Uniforme 1 75.366 beta_th Champ_Uniforme 1 0 }"
    dico["dispersed_phase"] = "gaz_air Fluide_Incompressible { mu champ_uniforme 1 1.85e-5 rho champ_uniforme 1 1.2 lambda Champ_Uniforme 1 0.023 Cp Champ_Uniforme 1 1006 beta_th Champ_Uniforme 1 0 }"
    dico["frottement_interfacial"] = "Tomiyama { contamination 2 }"
    dico["masse_ajoutee"] = "coef_constant { }"
    dico["portance_interfaciale"] = "Tomiyama { }"
    dico["dispersion_bulles"] = "turbulente_burns { }"
    dico["WLu"] = 'paroi_frottante_loi { }'
    dico["beta_portance"] = 1
    dico["beta_disp"] = 1
    dico["beta_wall_disp"] = 1
    dico["beta_wall_lift"] = 1
    return dico
    
def dico_trial_matrix(row):
    dico = {}
    dico["diametre_bulles"] = row["Dbubble"]
    dico["u_0"] = row["u_0"]
    dico["grav"] = row["gravity"]
    dico["alpha_l0"] = row["alpha_l0"]
    dico["alpha_v0"] = row["alpha_g0"]
    dico["CI_k"] = row["CL_k"]
    dico["CI_diss"] = row["CL_om"]
    dico["h_sonde"] = str(row["height"])
    dico["x_sonde"] = str(row["D_h"]/2*numpy.cos(2.5*numpy.pi/180))
    dico["y_sonde"] = str(row["D_h"]/2*numpy.sin(2.5*numpy.pi/180))
    return dico

def dico_turbulence_model(model_name, row):
    dico = {}
    if model_name == "komega" or model_name == "komega-sato":
        pass
    elif model_name == "komega-hzdr":
        dico["CeHZDR"] = 1.0
        dico["CkHZDR"] = 0.002
    elif model_name == "komega-wit" or model_name == "komega-wit-wif":
        d_b = 1e-3
        u_r = numpy.sqrt(row["alpha_g0"] * d_b * (rho_l - rho_g) * abs(grav) / rho_l)
        dico["ICkWIT"] = 0.1*u_r**2
        dico["BCfrontWIT"] = 0.1*u_r**2
    return dico

Create meshes from salomé library. Scripts are avalaible in meshes directory.

In [ ]:
run.useMEDCoupling()
if force_computation:
    from src.build_meshes import build_mesh_colin_1, build_mesh_colin_2
    os.chdir(f"{build}/")
    build_mesh_colin_1(5.0, ".", "mesh_Colin_1.med")
    build_mesh_colin_2(5.0, ".", "mesh_Colin_2.med")
    os.chdir("../")

## Runs submissions

In [ ]:
# Launch calculations
if force_computation or not os.path.exists(build):
    dmodel = dico_model()
    for index, row in trial_matrix.iterrows(): # all experiment case
        if index in subset:
            dico_tm = dico_trial_matrix(row)
            for turbmod in turbulence_choice: # all turbulence models
                dturb = dico_turbulence_model(turbmod, row)
                for ref in refinement_levels: # all refinements level
                    casename = f"Colin2012_{row['SetNumber']}_{turbmod}/m{ref}"

                    dico = {}
                    
                    dico["name_mesh"] = f"../../mesh_Colin_{ref}.med"
    
                    dd = {**dparam, **dmodel, **dico_tm, **dturb, **dico}
    
                    myrun = run.addCaseFromTemplate(f"{dataroot}-{turbmod}.data",
                                                    casename,
                                                    dd,
                                                    nbProcs=number_of_partitions)
                    if number_of_partitions > 1:
                        myrun.partition()
    run.printCases()
else:
    print('No recalculation, reusing old results!')

Tableau des performance : décommenter la dernière ligne pour afficher le tableau

In [ ]:
if force_computation:
    run.runCases()

In [ ]:
table = run.tablePerf()
table.index = [aaa.split("/")[0] for aaa in table.index]
table

## Plot residuals

In [ ]:
from trustutils import visit
from trustutils.jupyter import plot
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.close()
fig = plt.figure(figsize = (16, 4))
color_raf = ["red", "green", "blue"]
name_phys = [["vitl", "vitg"], 
             ["alphal", "alphag"],  
             ["k"], 
             ["diss"]]
par = "PAR_"
if number_of_partitions == 1 : 
    par = ""

axs = fig.subplots(1, 4, sharex=True, sharey=True)

for ii, nn in enumerate(name_phys):
    for jj, row in trial_matrix.iterrows():
        if jj in subset: # only the selected cases
            for turbmod in turbulence_choice:
                for rr in refinement_levels:
                    repo = f"Colin2012_{row['SetNumber']}_{turbmod}/m{rr}"
                    residuals = pandas.read_csv(f"{build}/{repo}/{par}{dataroot}-{turbmod}.dt_ev", sep="\t")
                    residuals = residuals.iloc[:,:13]
                    residuals.columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg', 'alphal', 'alphag', 'Tl', 'Tg', 'diss', 'k']               
                    label_loc = repo
                    for ll, vv in enumerate(nn) :
                        axs[ii].plot(residuals["time"][::], residuals[vv][::], "-", 
                                     label=label_loc, color=color_raf[rr])

    axs[ii].set_xlim(0, max(residuals["time"]))
    axs[ii].set_yscale("log")
    axs[ii].set_title(f"convergence {nn[0]}")
    axs[ii].set_xlabel('time')
    axs[ii].set_ylabel(f"convergence {nn[0]}")

plt.tight_layout()
if save_figures:
    plt.savefig(f"{build}/convergence.pdf")
plt.show()

### Comparaison des résultats de simulation aux données expérimentales
Stocker les résultats des simulations dans une liste

In [ ]:
from pathlib import Path

def loadText(data, index_column=0, nb_column=-1, transpose=True, dtype="float", skiprows=0):
        
    if nb_column == -1:
        nb = None
    else:
        nb = index_column + nb_column
    try:
        if transpose:
            matrix = numpy.loadtxt(f'{data}', dtype=dtype, skiprows=skiprows).T[index_column:nb]
        else:
            matrix = numpy.loadtxt(data, dtype=dtype, skiprows=skiprows)[index_column:nb]
    except:
        matrix = numpy.loadtxt(data, dtype=dtype, skiprows=skiprows)            
    return matrix

print("Warning: post-traitment perfomr for only one turbulence model !")
turbmod = turbulence_choice[0] 

# Recover simulation results in a list
# sim is a table were results are stored for every probe, every mesh and every experiment
sim = [ [ -1 for r in range(len(refinement_levels)) ] for n in range(n_exp) ]
# every element in sim will be a dataframe (pandas) with results from every probe as columns
sonde_gradv = sonde_k_WIT = sonde_epsilon_WIT = sonde_F = sonde_BIA = True

for n, row in trial_matrix.iterrows():
    if n in subset: # only the selected cases
        for r, refinenement in enumerate(refinement_levels):
            repo = f"{build}/Colin2012_{row['SetNumber']}_{turbmod}/m{refinenement}"
            name = f"{dataroot}-{turbmod}"
            
            sim[n][r] = pandas.DataFrame()
            sim[n][r]['r+']= numpy.linspace(0,1, 100)
            sim[n][r]['dp'] =numpy.array(loadText(f"{repo}/{par}{name}_DP.son"))[1::,-1]
            # vitesses
            sim[n][r]['vxl']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_EAU.son"))[1::3,-1]
            sim[n][r]['vyl']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_EAU.son"))[2::3,-1]
            sim[n][r]['vzl']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_EAU.son"))[3::3,-1]
            sim[n][r]['vxg']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_AIR.son"))[1::3,-1]
            sim[n][r]['vyg']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_AIR.son"))[2::3,-1]
            sim[n][r]['vzg']=numpy.array(loadText(f"{repo}/{par}{name}_VITESSE_AIR.son"))[3::3,-1]
            # bulles
            sim[n][r]['DB'] =1000*numpy.array(loadText(f"{repo}/{par}{name}_DIAMETRE.son"))[2::2,-1]
            sim[n][r]['alp']=numpy.array(loadText(f"{repo}/{par}{name}_ALPHA_AIR.son"))[1::,-1]
            # turbulence
            sim[n][r]['k_SIT']=numpy.array(loadText(f"{repo}/{par}{name}_K.son"))[1::,-1]
            sim[n][r]['omega_SIT']=numpy.array(loadText(f"{repo}/{par}{name}_DISS.son"))[1::,-1]
            sim[n][r]['nu_t']=numpy.array(loadText(f"{repo}/{par}{name}_VISCOSITE.son"))[1::,-1]
            # qdm
            sim[n][r]['conv']=numpy.array(loadText(f"{repo}/{par}{name}_CONV_V.son"))[1::6,-1]
            sim[n][r]['diff']=numpy.array(loadText(f"{repo}/{par}{name}_DIFF_V.son"))[1::6,-1]
            
            # if a probe for gradV is there
            try:
                sim[n][r]['dvxl_dx']=numpy.array(loadText(f"{repo}/{par}{name}_GRADV.son"))[1::9,-1]
                sim[n][r]['dvyl_dy']=numpy.array(loadText(f"{repo}/{par}{name}_GRADV.son"))[4::9,-1]
                sim[n][r]['dvzl_dz']=numpy.array(loadText(f"{repo}/{par}{name}_GRADV.son"))[9::9,-1]
                sim[n][r]['dvzl_dx']=numpy.array(loadText(f"{repo}/{par}{name}_GRADV.son"))[7::9,-1]
                sim[n][r]['upvp'] = sim[n][r]['nu_t']*numpy.abs(sim[n][r]['dvzl_dx']) # frottement turbulent u'v'
            except:
                sim[n][r]['dvxl_dx']=numpy.zeros(100)
                sim[n][r]['dvyl_dy']=numpy.zeros(100)
                sim[n][r]['dvzl_dz']=numpy.zeros(100)
                sim[n][r]['dvzl_dx']=numpy.zeros(100)
                sim[n][r]['upvp'] =numpy.zeros(100)
                sonde_gradv = False
                
            # if du Cluzeau's model is used
            try:
                sim[n][r]['k_WIT']  =numpy.array(loadText(f"{repo}/{par}{name}_K_WIT.son"))[1::,-1]
            except:
                sim[n][r]['k_WIT']  =numpy.zeros(100)
                sonde_k_WIT = False
                
            try:
                sim[n][r]['epsilon_WIT']  =numpy.array(loadText(f"{repo}/{par}{name}_DISS_K_WIT.son"))[1::,-1]
            except:
                sim[n][r]['epsilon_WIT']  =numpy.zeros(100)
                sonde_epsilon_WIT = False
                
                
            # if interfacial forces are in sondes
            try:
                sim[n][r]['drag']=numpy.array(loadText(f"{repo}/{par}{name}_DRAG.son"))[1::6,-1] # composante selon x - phase liquide
                sim[n][r]['lift']=numpy.array(loadText(f"{repo}/{par}{name}_LIFT.son"))[1::6,-1] # composante selon x - phase liquide
                sim[n][r]['disp']=numpy.array(loadText(f"{repo}/{par}{name}_DISP.son"))[1::6,-1] # composante selon x - phase liquide
                sim[n][r]['lub']=numpy.array(loadText(f"{repo}/{par}{name}_LUB.son"))[1::6,-1]   # composante selon x - phase liquide
            except:
                sim[n][r]['drag']=numpy.zeros(100)
                sim[n][r]['lift']=numpy.zeros(100)
                sim[n][r]['disp']=numpy.zeros(100)
                sim[n][r]['lub']=numpy.zeros(100)
                sonde_F = False
                
            # if BIA source term is in sondes : div(-Rij_BIA)
            try:
                sim[n][r]['BIF']=numpy.array(loadText(f"{repo}/{par}{name}_BIF.son"))[1::6,-1] # composante selon x - phase liquide
            except:
                sim[n][r]['BIF']=numpy.zeros(100)
                sonde_BIA = False
    
            # Calcul de k_tot (SIT+BIA)
            sim[n][r]['ur'] = numpy.sqrt( (sim[n][r]['vxg'] - sim[n][r]['vxl'])**2 + (sim[n][r]['vyg'] - sim[n][r]['vyl'])**2 + (sim[n][r]['vzg'] - sim[n][r]['vzl'])**2)
            sim[n][r]['k_tot'] = sim[n][r]['k_SIT'] 
            if "WIT" in turbulence_choice:
                sim[n][r]['k_tot'] += sim[n][r]['k_WIT']
            if "WIF" in turbulence_choice:
                sim[n][r]['k_WIF'] = (9/20 + (1/20+3/2*0.25)) * sim[n][r]['alp'] * sim[n][r]['ur']**2
                sim[n][r]['k_tot'] += sim[n][r]['k_WIF']
            else:
                sim[n][r]['k_WIF']=numpy.zeros(100)

# Warning if some results are not available (by default we put 0 instead)
if not(sonde_gradv):
    print("Warning: velocity gradients are not available in the probes!")
if not(sonde_k_WIT):
    print("Warning: k_WIT gradients are not available in the probes!")
if not(sonde_epsilon_WIT):
    print("Warning: epsilon_WIT gradients are not available in the probes!")
if not(sonde_F):
    print("Warning: Interfacial forces are not available in the probes!")
if not(sonde_BIA):
    print("Warning: The BIA source term in momentum is not available in the probes!")

Retrieve experimental data into a list

In [ ]:
# # recover test results
Exp = [ {} for n in range(n_exp) ]

folder_path = f"{build}/../data_exp/"
            
for n, row in trial_matrix.iterrows():
    prefix = row['SetNumber']
    for filename in os.listdir(folder_path):
        if filename.startswith(prefix):
            df = pandas.read_csv(folder_path + filename, sep="\t")
            new_columns = df.iloc[2]
            df.columns = new_columns
            # Delete rows 0, 1, and 3
            rows_to_delete = [0, 1, 2, 3]
            df = df.drop(rows_to_delete)
            df.columns = [str(col) for col in df.columns]
            #df = df.fillna(0)
            df = df.astype(float)
            if 'G' in filename:
                Exp[n]['G'] = df
            elif 'L' in filename:
                Exp[n]['L'] = df
            elif 'Alp' in filename:
                Exp[n]['Alp'] = df
            else:
                print(f"There is a problem, the file {filename} does not comply with the nomenclature.")



Tracé des différents profils ainsi que les données expérimentales correspondantes

Attention : seuls les mesures des fluctuations axiales sont disponibles, il est donc impossible de reconstruire l'énergie cinétique turbulente à partir des données expérimentales. On utilise alors l'approximation $u_L^\prime=2v_L^\prime=2w_L^\prime$ pour déduire $u_L^\prime$ à partir de $k$ obtenu par simulation numérique.
$k=1/2(u_L^{\prime2}+v_L^{\prime2}+w_L^{\prime2})=1/2(\frac{3}{2}u_L^{\prime2})=\frac{3}{4}u_L^{\prime2}$ donc $u_L^\prime=\sqrt{\frac{4}{3}k}$

In [ ]:
fig = plt.figure(figsize = (15, 15))
axs = fig.subplots(7, 4 )

# mesh refinement
ref = refinement_levels[0]
# marker size for experimental data
ms = 3
# couleurs plot
tab_color = ["saddlebrown", "red",  "orange", "chartreuse", "forestgreen", "lightseagreen", 'cyan', "royalblue",  "indigo", 'purple', 'magenta']
# many experiments can be grouped using the liquid flow rate J_L
# index corresponding to each group
flow = [[i for i in range(3)],     # single phase
       [i for i in range(3, 8)],   # upwards
       [i for i in range(8, 14)],  # downwards
       [i for i in range(14,17)]]  # micro
# Label used in the plot for each group
label_line = ["Colin single-phase",
              "Colin upwards",
              "Colin downwards",
              "Colin microgravity"]

# axis limiters
lim_alp = [.15,.15,.15,.15]
lim_u = [1.5,1.5,1.5,1.5]
lim_u_min = [0,0,0,0]
lim_u_prime = [.2,.2,.2,.2]
lim_d = [2,6,8,2.5]

# plot results    
for i in range(len(flow)): # values of J_L
    for n in flow[i]:      # experiments with the same J_L

        prefix = 'not_found'
        for index, row in trial_matrix.iterrows(): # all experiment case
            if index == n:
                prefix = row['SetNumber']
        if prefix == 'not_found':
            raise ValueError("Prefix is not found")

        if n in subset:
            # Simulation results
            axs[0,i].plot([-1, -2],[-1, -1], "o-", label = f"{prefix.replace('_', ' ')}", color=tab_color[n-flow[i][0]], markerfacecolor = "White")
            axs[1,i].plot(sim[n][ref]['r+'], sim[n][ref]['alp'], color=tab_color[n-flow[i][0]]) # alpha(r/R) : void fraction
            axs[2,i].plot(sim[n][ref]['r+'], sim[n][ref]['DB'], color=tab_color[n-flow[i][0]])  # d_B(r/R)   : bubble diameter
            axs[3,i].plot(sim[n][ref]['r+'], sim[n][ref]['vzl'], color=tab_color[n-flow[i][0]]) # U_L(r/R)   : mean liquid velocity
            axs[4,i].plot(sim[n][ref]['r+'], sim[n][ref]['vzg'], color=tab_color[n-flow[i][0]]) # U_G(r/R)   : mean air velocity
            
            if i<3: # single phase : hyp TI (isotrope)
                axs[5,i].plot(sim[n][ref]['r+'], numpy.sqrt(2/3*sim[n][ref]['k_tot']), color=tab_color[n-flow[i][0]]) # u'_L(r/R) : liquid velocity fluctuations (computed as sqrt(k))    
            else:
                axs[5,i].plot(sim[n][ref]['r+'], numpy.sqrt(4/3*sim[n][ref]['k_tot']), color=tab_color[n-flow[i][0]]) # u'_L(r/R) : liquid velocity fluctuations (computed as sqrt(k))
            
        # Experimental points
        if ("G" in Exp[n]):
            if ("a" in Exp[n]["G"]):
                axs[1,i].plot(Exp[n]["G"]['r+'], Exp[n]["G"]['a'], "o", color=tab_color[n-flow[i][0]], markersize = ms ,markerfacecolor = "White")
                axs[4,i].plot(Exp[n]["G"]["r+"], Exp[n]["G"]["UG"] , "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")
                axs[6,i].plot(Exp[n]["G"]["r+"], Exp[n]["G"]["u'G"], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")                  
            if ("d32e" in Exp[n]["G"]) :
                axs[2,i].plot(Exp[n]["G"]["r+"], Exp[n]["G"]["d32e"], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")  
            elif ("d" in Exp[n]["G"]) :
                axs[2,i].plot(Exp[n]["G"]["r+"], Exp[n]["G"]["d"], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")  
        if ("L" in Exp[n]):
            axs[3,i].plot(Exp[n]["L"]['r+'], Exp[n]["L"]['UL'], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")   
            axs[5,i].plot(Exp[n]["L"]['r+'], Exp[n]["L"]["u'L"], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")
        if ("Alp" in Exp[n]):
                axs[1,i].plot(Exp[n]["Alp"]['r+'], Exp[n]["Alp"]['a'], "o", color=tab_color[n-flow[i][0]], markersize = ms, markerfacecolor = "White")
    
        # Titles
        axs[0,i].text(0,.1, label_line[i], ha = "center", va = "center", fontsize = 14)
        axs[0,i].legend(loc = 'lower center', fontsize = 10, ncol=2)
        axs[0,i].set_xlim(-.5,.5)
        axs[0,i].set_ylim(-.7,.3)
        axs[0,i].axis('off')
        
        # plot labels
        axs[1,i].set_ylabel(r"$\alpha$")
        axs[2,i].set_ylabel(r"$d_B$ (mm)")
        axs[3,i].set_ylabel(r"$U_L$ (ms$^{-1}$)")
        axs[4,i].set_ylabel(r"$U_G$ (ms$^{-1}$)")
        axs[5,i].set_ylabel(r"$u_L^\prime$ (ms$^{-1}$)")
        axs[6,i].set_ylabel(r"$u_G^\prime$ (ms$^{-1}$)")
        axs[6,i].set_xlabel(r"$r/R$")
        
        # axis limiters
        axs[1,i].set_ylim(0, lim_alp[i])
        axs[2,i].set_ylim(0, lim_d[i])
        axs[3,i].set_ylim(lim_u_min[i], lim_u[i])
        axs[4,i].set_ylim(lim_u_min[i], lim_u[i])
        axs[5,i].set_ylim(0, lim_u_prime[i])
fig.suptitle("Comparison of simulation results with the experimental data of Catherine Colin", fontsize = 20)
fig.tight_layout()
if save_figures:
    plt.savefig(f"{build}/results.pdf")